<a href="https://colab.research.google.com/github/FNeffati/PDS/blob/main/LTSM-NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from imblearn.over_sampling import SMOTE

In [5]:
#replace this line with fullset path
path = "/content/drive/MyDrive/Copy of fullSet.csv"

full_set = pd.read_csv(path, dtype={'status': str})
full_set.head()

,time,xrsa_flux,xrsb_flux,status
0,2017-02-07 00:00:00,2.949822e-08,7.072423e-08,NaN
1,2017-02-07 00:01:00,2.716477e-08,6.869706e-08,NaN
2,2017-02-07 00:02:00,2.996076e-08,6.933664e-08,NaN
3,2017-02-07 00:03:00,3.124979e-08,6.796333e-08,NaN
4,2017-02-07 00:04:00,3.095799e-08,6.928261e-08,NaN


In [6]:
# full_set['peak'] = np.where(full_set['status'] == 'EVENT_PEAK', 1, 0)
in_event = False
event_label = 0
labels = []

for status in full_set['status']:
    if status == 'EVENT_START':
        in_event = True
        event_label = 1
    elif status == 'EVENT_END':
        in_event = False
        event_label = 0
    labels.append(event_label)

full_set['flare'] = labels

full_set.head(10000)

,time,xrsa_flux,xrsb_flux,status,flare
0,2017-02-07 00:00:00,2.949822e-08,7.072423e-08,NaN,0
1,2017-02-07 00:01:00,2.716477e-08,6.869706e-08,NaN,0
2,2017-02-07 00:02:00,2.996076e-08,6.933664e-08,NaN,0
3,2017-02-07 00:03:00,3.124979e-08,6.796333e-08,NaN,0
4,2017-02-07 00:04:00,3.095799e-08,6.928261e-08,NaN,0
...,...,...,...,...,...
9995,2017-02-14 19:56:00,4.130439e-09,3.169877e-08,NaN,0
9996,2017-02-14 19:57:00,3.370558e-09,3.293717e-08,NaN,0
9997,2017-02-14 19:58:00,4.871447e-09,3.275998e-08,NaN,0
9998,2017-02-14 19:59:00,4.171957e-09,3.249245e-08,NaN,0


In [7]:


# Load data from CSV
df = full_set

# Separate the 'flare' column from the features
features = df[['xrsa_flux', 'xrsb_flux']]
labels = df['flare']

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

def create_dataset(data, labels, look_back=1):
    dataX, dataY = [], []
    for i in range(len(data)-look_back-1):
        a = data[i:(i+look_back)]
        dataX.append(a)
        dataY.append(labels[i + look_back])
    return np.array(dataX), np.array(dataY)

look_back = 10
trainX, trainY = create_dataset(scaled_features, labels, look_back)

#add oversampling technique here?

In [8]:
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 2))


In [9]:
model = Sequential()
model.add(LSTM(50, input_shape=(look_back, 2)))  # 50 LSTM units; 2 because we have xrsa_flux and xrsb_flux
model.add(Dense(1, activation='sigmoid'))  # binary output; use 'sigmoid' activation

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model.fit(trainX, trainY, epochs=5, batch_size=500, verbose=1)


Epoch 1/5
 2614/12081 [=====>........................] - ETA: 2:58 - loss: 0.2666 - accuracy: 0.9210